In [1]:
import pandas as pd
import numpy as np

In [117]:
excelFile = r'files/sw_train_data_sample_33214.xlsx'
sample_df = pd.DataFrame(pd.read_excel(excelFile))
print(f"{sample_df.shape}")
sample_df.head()

(2051, 11)


,code,name,kPattern,k,d,dif,dea,macd,open,short,long
0,801012,农产品加工II,33214,54.193490,53.001464,4.653048,28.310486,-23.657439,-0.019959,4.203308,1.942705
1,801012,农产品加工II,33214,42.115690,53.469651,-8.987499,-17.455907,8.468408,0.199770,0.691922,0.101039
2,801012,农产品加工II,33214,13.509500,19.485156,-25.938599,-20.195254,-5.743346,0.360893,1.392745,0.721243
3,801012,农产品加工II,33214,64.128683,58.424104,-42.129105,-32.290747,-9.838358,0.432093,1.421845,0.847409
4,801012,农产品加工II,33214,85.045484,86.250993,-29.805827,-64.336337,34.530510,-0.170309,0.907151,0.225449


In [118]:
def set_0_or_1(x):
    if x > 0:
        return 1
    return 0

def percent_to_one(x):
    return int(x * 100) /100.0

def to_one_min_max_120(x):
    v = x / 120
    if  v > 1:
        v = 1
    if  v< -1:
        v =-1
    return int(v*100)/100.0

d = sample_df
d['d'] = d.k - d.d
d['d'] = (d.d /100).apply(percent_to_one)  #归一化
d['k'] = (d.k /100).apply(percent_to_one)  #归一化
d['open'] = d.open.apply(percent_to_one)   #归一化
d['macd'] = d.macd.apply(to_one_min_max_120)   #归一化
d['short'] = d.short.apply(set_0_or_1)
d['long'] = d.long * 10 + 0.5
sample_df = sample_df.drop(columns=['code','kPattern', 'name','dif','dea'])
sample_df.rename(columns={'k':'k_diff'},inplace=True)
sample_df.head()

,k_diff,d,macd,open,short,long
0,0.54,0.01,-0.19,-0.01,1,19.927053
1,0.42,-0.11,0.07,0.19,1,1.510390
2,0.13,-0.05,-0.04,0.36,1,7.712434
3,0.64,0.05,-0.08,0.43,1,8.974091
4,0.85,-0.01,0.28,-0.17,1,2.754486


In [119]:
import sklearn
from sklearn import model_selection

data  = sample_df.values
x,y=np.split(data,indices_or_sections=(4,),axis=1) #x为数据，y为标签
y = y[:,0:1].astype('int')  #取第一列
y

array([[1],
       [1],
       [1],
       ...,
       [1],
       [0],
       [1]])

In [120]:
train_data,test_data,train_label,test_label =model_selection.train_test_split(x,y, train_size=0.7,test_size=0.3)
train_label = train_label.ravel()
test_label = test_label.ravel()

In [121]:
from sklearn import svm
classifier=svm.SVC(C=2,kernel='rbf',gamma=10,decision_function_shape='ovr') # ovr:一对多策略
classifier.fit(train_data,train_label) #ravel函数在降维时默认是行序优先

print("训练集：",classifier.score(train_data,train_label))
print("测试集：",classifier.score(test_data,test_label))

训练集： 0.7149825783972126
测试集： 0.6931818181818182


,kPattern,k_diff,d,macd,open,short,long
0,46418,76.080320,18.173668,-4.023488,-0.034919,0.467475,0.033330
1,39776,82.990059,9.706241,-2.389732,-0.020127,0.237116,-0.194516
2,33215,86.664484,4.752863,-0.932882,-0.014234,0.127453,-0.193571
3,33215,91.173888,4.231078,0.055731,0.052279,1.463844,0.923712
4,33216,93.380019,2.973889,1.164693,-0.038353,0.239515,-0.158650


In [20]:
d = sample_df
sample_df['d'] = d.k - d.d
sample_df.head()

,code,name,kPattern,k,d,dif,dea,macd,open,short,long
0,801012,农产品加工II,46418,76.080320,18.173668,11.395251,15.418739,-4.023488,-0.349192,0.467475,0.033330
1,801012,农产品加工II,39776,82.990059,9.706241,11.625503,14.015235,-2.389732,-0.201269,0.237116,-0.194516
2,801012,农产品加工II,33215,86.664484,4.752863,11.312657,12.245539,-0.932882,-0.142343,0.127453,-0.193571
3,801012,农产品加工II,33215,91.173888,4.231078,11.253561,11.197830,0.055731,0.522793,1.463844,0.923712
4,801012,农产品加工II,33216,93.380019,2.973889,12.710930,11.546237,1.164693,-0.383528,0.239515,-0.158650
